### Análise Exploratória de Dados (EDA) e Sistema de Recomendação com dados do Spotify

#### Instalando Bibliotecas

In [ ]:
!pip install \
    numpy \
    pandas \
    seaborn \
    plotly \
    matplotlib \
    scikit-learn \
    setuptools \
    yellowbrick \
    wheel \
    scipy \
    spotipy #--break-system-packages

#### Importando as bibliotecas necessárias

In [54]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
#%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings("ignore")


#from yellowbrick.target import FeatureCorrelation # Continuar olhando por essa biblioteca não funciona


#### Carregando os dados

In [55]:
data = pd.read_csv("dataset/data.csv")
genre_data = pd.read_csv("dataset/data_by_genres.csv")
year_data = pd.read_csv("dataset/data_by_year.csv")
artist_data = pd.read_csv("dataset/data_by_artist.csv")

#### Lendo as primeiras linhas do dataset

In [ ]:
data.head()

In [ ]:
genre_data.head()

In [ ]:
year_data.head()

In [ ]:
artist_data.head()

#### Verificando os registros nulos por coluna

In [ ]:
pd.isnull(data).sum()

In [ ]:
pd.isnull(genre_data).sum()

In [ ]:
pd.isnull(year_data).sum()

In [ ]:
pd.isnull(artist_data).sum()

#### Resumo do dataframe (Coluna, Datatype e tamanho)

In [ ]:
data.info()

In [ ]:
genre_data.info()

In [ ]:
year_data.info()

In [ ]:
artist_data.info()

#### Estatísticas descritivas das variáveis numéricas presentes nas colunas

In [ ]:
data.describe().transpose()

In [ ]:
# Estatísticas descritivas das variáveis numéricas presentes nas colunas
genre_data.describe().transpose()

In [ ]:
# Estatísticas descritivas das variáveis numéricas presentes nas colunas
year_data.describe().transpose()

In [ ]:
artist_data.describe().transpose()

### Top 5 músicas menos populares

In [ ]:
least_popular = data.sort_values('popularity', ascending = True).head(5) 
least_popular[['year','name','release_date','artists']]

### Top 5 músicas mais populares

In [ ]:
most_popular = data.sort_values('popularity', ascending = False).head(5) 
most_popular[['year','name','release_date','artists']]

### Top 10 artistas mais populares

Quanto maior o seu índice de popularidade, mais provável é que o algoritmo recomende suas músicas para novos ouvintes e te coloque em playlists algorítmicas como "Radar de Novidades" e "Descobertas da Semana".

In [ ]:
popular_artists = artist_data.sort_values('popularity', ascending=False).head(10)
popular_artists[['popularity','energy','artists','danceability']]

#### Convertendo a duração da música de milissegundos para minutos

In [ ]:
data['duration']=data['duration_ms'].apply(lambda x : round((x/60000),2))
genre_data['duration']=genre_data['duration_ms'].apply(lambda x : round((x/60000),2))
year_data['duration']=year_data['duration_ms'].apply(lambda x : round((x/60000),2))
artist_data['duration']=artist_data['duration_ms'].apply(lambda x : round((x/60000),2))
data.duration.head()

###     Correlação entre Features e Popularidades

In [ ]:
# As features que você já definiu
feature_names = ['acousticness', 'danceability', 'energy', 'instrumentalness',
                 'liveness', 'loudness', 'speechiness', 'tempo', 'valence',
                 'duration', 'explicit']

# Separando os dados X e y
X, y = data[feature_names], data['popularity']

# Calculando correlações entre cada feature e o target
corr_matrix = pd.concat([X, y], axis=1).corr()

# Visualizando o mapa de calor de correlações com seaborn
plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlação entre Features e Popularidade')
plt.show()


In [77]:
# feature_names = ['acousticness', 'danceability', 'energy', 'instrumentalness',
#        'liveness', 'loudness', 'speechiness', 'tempo', 'valence','duration','explicit']

# X, y = data[feature_names], data['popularity']

# # Create a list of the feature names
# features = np.array(feature_names)

# # Instantiate the visualizer
# visualizer = FeatureCorrelation(labels=features)

# visualizer.fit(X, y)     # Fit the data to the visualizer
# visualizer.show();        # Finalize and render the figure

In [ ]:
# Select only numeric columns
numeric_cols = data.select_dtypes(include=[float, int])

# Calculate the correlation matrix
corr_df = numeric_cols.corr(method='pearson')

# Plot the heatmap
plt.figure(figsize=(14,8))
heatmap = sns.heatmap(corr_df, annot=True, fmt='.1g', vmin=-1, vmax=1, center=0, cmap="BrBG", linewidths=2, linecolor="Black")
heatmap.set_title('Mapa de calor de correlação entre variáveis')
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=90)
plt.show()


### Gráfico de regressão entre Intensidade (Loudness) e Energia

In [ ]:
sns.regplot(data=data, y='loudness', x='energy', scatter_kws={"color": "violet", 's':2}, marker='2', line_kws={"color": "black", 'linewidth':1.5}).set(title='Correlação (Loudness VS Energy)')

### Gráfico de regressão entre Popularidade e Acústica (Acousticness)

In [ ]:
sns.regplot(data=data, y='popularity', x='acousticness', scatter_kws={"color": "c", 's':3}, marker='X', line_kws={"color": "black", 'linewidth':1.5}).set(title='Correlação (Popularity VS Acousticness)');

### Gráfico de regressão entre Discurso (Speechiness) e Acústica (Acousticness).

In [ ]:
sns.regplot(data=data, y='speechiness', x='acousticness', scatter_kws={"color": "goldenrod", 's':3}, marker='+', line_kws={"color": "black", 'linewidth':1.5}).set(title='Correlação (Speechiness VS Acousticness)');

### Gráfico de regressão entre Popularidade e Dançabilidade (Danceability)

In [ ]:
sns.regplot(data=data, y='popularity', x='danceability', scatter_kws={"color": "red", 's':3}, marker='*', line_kws={"color": "black", 'linewidth':1.5}).set(title='Correlação (Popularity VS Danceability)');

### Gráfico de barras para visualizar a correlação entre a duração das músicas e seus diferentes gêneros


In [ ]:
top_genres = genre_data.nlargest(20, 'popularity')
plt.title("Duração das Músicas em Diferentes Gêneros")
#sns.color_palette("rocket", as_cmap= True)
sns.barplot(y='genres', x='duration', data=top_genres, color='lightsalmon')
plt.xlabel ("Duration in seconds")
plt.ylabel("Genres");

## Música ao Longo do Tempo

### Gráfico de barras para visualizar a duração das músicas ao longo dos anos.

In [ ]:
plt.figure(figsize=(20,6))
sns.barplot(x="year",y="duration", errwidth=False, data=data).set(title='Year Vs Duration')
plt.xticks(rotation=90);

### Gráfico de distribuição para visualizar o número total de músicas em cada ano desde 1921 em nosso banco de dados do Spotify.

In [ ]:
fig = px.histogram(data, x="year")
fig.show()


### Como o som geral da música mudou de 1921 a 2020

In [ ]:
def extract_decade(year):
    start_period = int(year/10) * 10
    decade = '{}s'.format(start_period)
    return decade

data['decade'] = data['year'].apply(extract_decade)

sns.set(rc={'figure.figsize':(10 ,5)})
sns.countplot(data['decade'])

In [ ]:
sound_features = ['acousticness', 'liveness', 'instrumentalness', 'energy', 'danceability', 'valence']
fig = px.line(year_data, x='year', y=sound_features)
fig.show()

### Características de vários gêneros ao longo dos anos
Usando as características de áudio de diferentes gêneros, podemos compará-los e identificar suas diferenças únicas na música.

In [ ]:
top_genres = genre_data.nlargest(10, 'popularity')

fig = px.bar(top_genres, x='genres', y=['valence', 'energy', 'danceability', 'acousticness'], barmode='group')
fig.show()

### Agrupando Gêneros com K-Means.

Usando o algoritmo simples de clustering K-means para dividir os gêneros neste conjunto de dados em dez clusters com base nas características numéricas de áudio de cada gênero.

In [89]:
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

cluster_pipeline = Pipeline([('scaler', StandardScaler()), ('kmeans', KMeans(n_clusters=10))])
X = genre_data.select_dtypes(np.number)
cluster_pipeline.fit(X)
genre_data['cluster'] = cluster_pipeline.predict(X)

In [ ]:
from sklearn.manifold import TSNE

tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=False))])
genre_embedding = tsne_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
projection['genres'] = genre_data['genres']
projection['cluster'] = genre_data['cluster']

fig = px.scatter(projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'genres'])
fig.show()

### Agrupando Músicas com K-Means

In [91]:
number_cols = ['valence', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo']
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), ('kmeans', KMeans(n_clusters=20, verbose=False))], verbose=False)

X = data[number_cols].select_dtypes(np.number)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)
data['cluster_label'] = song_cluster_labels

In [ ]:
from sklearn.decomposition import PCA

pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2))])
song_embedding = pca_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=song_embedding)
projection['title'] = data['name']
projection['cluster'] = data['cluster_label']

fig = px.scatter(projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'title'])
fig.show()

### Construindo o Sistema de Recomendação
A análise e as visualizações deixam claro que gêneros musicais semelhantes tendem a se agrupar, assim como músicas com características similares aparecem próximas no espaço de dados.

Esse comportamento faz sentido, já que gêneros parecidos compartilham influências e contextos históricos, e o mesmo acontece com as músicas dentro desses gêneros. Essa proximidade nos dados pode ser explorada para criar um sistema de recomendação eficaz.

Podemos utilizar as músicas que um usuário já ouviu para identificar padrões e, com base na similaridade dos dados, sugerir novas músicas com características próximas às suas preferências.

Para implementar esse sistema, utilizaremos o Spotipy, um cliente Python para a API Web do Spotify, que facilita a busca e consulta ao catálogo musical. Com ele, conseguiremos acessar as informações necessárias para construir recomendações personalizadas de forma eficiente.

In [93]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict
import pandas as pd
from dotenv import load_dotenv

# Carrega as variáveis do arquivo .env
load_dotenv()

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIFY_CLIENT_ID"),
                                                           client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")))

def find_song(name=None, artist=None):
    song_data = defaultdict()

    if name:
        results = sp.search(q=f'track:{name}', limit=1)
    elif artist:
        results = sp.search(q=f'artist:{artist}', limit=1)
    else:
        return None

    if results['tracks']['items'] == []:
        return None

    results = results['tracks']['items'][0]
    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]

    song_data['name'] = [results['name']]
    song_data['artist'] = [results['artists'][0]['name']]
    song_data['explicit'] = [int(results['explicit'])]
    song_data['duration_ms'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]
    song_data['year'] = [results['album']['release_date'][:4]]

    for key, value in audio_features.items():
        song_data[key] = value

    return pd.DataFrame(song_data)


In [94]:
def get_song_data(song, spotify_data):
    if 'name' in song:
        try:
            song_data = spotify_data[spotify_data['name'] == song['name']].iloc[0]
            return song_data
        except IndexError:
            return find_song(name=song['name'])
    elif 'artist' in song:
        try:
            song_data = spotify_data[spotify_data['artists'].apply(lambda x: song['artist'].lower() in x.lower())]
            return song_data
        except IndexError:
            return find_song(artist=song['artist'])
    else:
        return None

def get_mean_vector(song_list, spotify_data):
    song_vectors = []
    
    for song in song_list:
        song_data = get_song_data(song, spotify_data)
        
        if song_data is None or song_data.empty:
            print(f"Warning: {song.get('name', song.get('artist', 'Unknown'))} does not exist in Spotify or in database")
            continue
        
        try:
            if isinstance(song_data, pd.DataFrame):
                song_vector = song_data[number_cols].mean().values  
            else:
                song_vector = song_data[number_cols].values  
            song_vectors.append(song_vector)
        except KeyError as e:
            print(f"Erro ao tentar obter características da música {song.get('name', song.get('artist', 'Unknown'))}: {e}")
            continue
    
    if len(song_vectors) == 0:
        print("Nenhuma música válida encontrada.")
        return None

    try:
        song_matrix = np.array(song_vectors)
        if not check_vector_lengths(song_vectors):
            print("Os vetores de músicas têm comprimentos diferentes.")
            return None
        return np.mean(song_matrix, axis=0)
    except ValueError as e:
        print("Erro ao tentar criar a matriz: ", e)
        return None

import pandas as pd

def check_vector_lengths(vectors):
    lengths = [len(v) for v in vectors]
    return all(length == lengths[0] for length in lengths)

def flatten_dict_list(dict_list):
    flattened_dict = defaultdict(list)
    for dictionary in dict_list:
        for key, value in dictionary.items():
            flattened_dict[key].append(value)
    return flattened_dict

def recommend_songs(song_list, spotify_data, n_songs=5):
    metadata_cols = ['name', 'year', 'artists']
    song_dict = flatten_dict_list(song_list)
    
    song_center = get_mean_vector(song_list, spotify_data)
    
    if song_center is None:
        print("Não foi possível calcular o vetor médio para as músicas fornecidas.")
        return []
    
    scaler = song_cluster_pipeline.steps[0][1]
    scaled_data = scaler.transform(spotify_data[number_cols])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))
    
    distances = cdist(scaled_song_center, scaled_data, 'cosine')
    index = list(np.argsort(distances)[:, :n_songs][0])
    
    rec_songs = spotify_data.iloc[index]
    
    if 'name' in song_dict:
        rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
    
    df_rec_songs = rec_songs[metadata_cols].to_dict(orient='records')
    df_resultados = pd.DataFrame(df_rec_songs)
    
    df_resultados = df_resultados.rename(columns={
        'artists': 'Artista/Banda',
        'name': 'Musica',
        'year': 'Ano'
    })
    
    return df_resultados




#### Recommended Songs:

In [ ]:
recommend_songs([{'name': 'Everybody Wants To Rule The World'}], data)

In [ ]:
recommend_songs([{'artist': 'Avenged Sevenfold'}], data)